# Importazione librerie e struttura dati iniziale 

In [1]:
import nltk 
import os
import re
import numpy as np # Per espressioni matematiche
import emoji
import matplotlib.pyplot as plt # Per rappresentazione di funzioni matematiche lato grafico

def convert(char_list):
    converted = ""
    for char in char_list:
        converted += str(char)
    return converted

# Processamento input, hashtag e emoticons

In [9]:
df = 'row_data/dataframes/'

emotions = {
    'anger': [],
    'anticipation': [],
    'disgust': [],
    'fear': [],
    'joy': [],
    'sadness': [],
    'surprise': [],
    'trust': []  
}

hashtags = {}
emojis = {}

# Lista cotenente le chiavi del dictionary emotions, per facilitare gli accessi
emotions_keys = emotions.keys()

# Accesso al file e lettura delle singole linee
for file_name in os.listdir(df):
    with open((df + file_name), 'r') as emotion_df:       
        emotion_content = emotion_df.readlines()
        line_list = []
        
        # Inserimento dei singoli tweet nel file + pulitura input
        for emotion_line in emotion_content:
            emotion_line = emotion_line.replace('USERNAME','').replace('URL','').strip()
            line_list.append(emotion_line)
            
            # Processamento Hashtags
        
            # Se '#' è presente nel tweet su cui si sta iterando si 
            # trova la posizione di '#' e si crea una lista di caratteri 
            # che verrà convertita
            if '#' in emotion_line:
                hashtag_prefix_position = emotion_line.find('#')
                hashtag_end_position = emotion_line.find(' ', hashtag_prefix_position) # Cercherà il primo spazio a partire dalla posizione di #, serve per rilevare la dell'hashtag
                char_list = [emotion_line[char] for char in range(hashtag_prefix_position,hashtag_end_position)]
                hashtag = convert(char_list)
                if hashtag != '':
                    if not hashtag in hashtags:
                        hashtags[hashtag] = 1
                    else:
                        hashtags[hashtag] += 1
        
        
            # Processamento Emojis
        
            # Se un'emoji è presente nella voce di emotions su cui si sta iterando la si mette nel dictionary apposito
            for char in emotion_line:
            
                # Si confronta il carattere dell'iterazioni 
                # corrente con il dictionary emoji.UNICODE_EMOJI 
                # (che contiene l'emoji: :nomeemoji:), 
                # andava aggiunto l'accesso alle sole variabili inglesi
                # altrimenti il programma non sarebbe andato
                if char in emoji.UNICODE_EMOJI['en']:
                    if not char in emojis:
                        emojis[char] = 1
                    else:
                        emojis[char] += 1
            
    # Inserimento di della lista line_list nel dictionary emotions
    for key in emotions_keys:
        if key in file_name:
            emotions[key] = line_list


print(emotions['anger'][2])
print(hashtags)
print(emojis)

this weeken d: tcu football , next weeken d: marriage  @ amon g . carter stadium
{'#sotakeoffallyourclothes': 2, '#bittersweet': 34, '#deadphone': 2, '#super': 8, '#windows': 1, '#lowkey': 10, '#truth': 66, '#wahh': 50, '#tiredasfuck': 2, '#1d': 10, '#hurting': 5, '#mysisterskeeper': 1, '#fml': 685, '#ihatebeingsick': 4, '#incompletetrio': 1, '#thecolony': 1, '#oomf': 1524, '#jk': 30, '#imtrying': 2, '#bamaloss': 1, '#family': 124, '#twatter': 1, '#rainraingoaway': 1, '#nj': 1, '#intensity': 1, '#weneedfriends': 1, '#mytruth': 10, '#heartbreak': 5, '#ouch': 76, '#sad': 99, '#wrestlerreflex': 1, '#rit': 1, '#confused': 80, '#poophead': 1, '#awk': 132, '#dying': 62, '#crybaby': 4, '#goodfriendstatus': 1, '#peace': 6, '#sadtweet': 336, '#sadness': 6, '#depressed': 62, '#pleasehelp': 3, '#soproud': 11, '#welost': 1, '#imissyou': 31, '#jeremyscott': 1, '#whatalife': 4, '#scared': 165, '#bubbagump': 1, '#mlsplayoffs': 1, '#fuck': 91, '#girlsnight': 19, '#sometimesiwonder': 93, '#sadlife': 13